In [5]:
from FinMind.data import DataLoader
from dotenv import load_dotenv
import os

token = os.getenv("API_TOKEN")

api = DataLoader()
api.login_by_token(api_token=token)

In [ ]:
df = api.taiwan_stock_daily(
    stock_id='2330',
    start_date='202-04-02',
    end_date='2020-04-12'
)